# Exportation des DataFrames pour intégration MySql# P3_Réalisez une étude de santé publique

In [339]:
#impotter des librairies
import pandas as pd
import numpy as np
#impotter de librairie sqlite3 et Créer une base de données


In [340]:
#importer des database
df_population = pd.read_csv('fr_population.csv')
df_animaux = pd.read_csv('fr_animaux.csv')
df_vegetaux = pd.read_csv('fr_vegetaux.csv')
df_céréales = pd.read_csv('fr_céréales.csv')
df_sousalimentation = pd.read_csv('fr_sousalimentation.csv')

In [341]:
#Supprimer la ligne Chine
df_population = df_population[df_population['Code zone'] != 351]
df_animaux = df_animaux[df_animaux['Code zone'] != 351]
df_vegetaux = df_vegetaux[df_vegetaux['Code zone'] != 351]
df_céréales = df_céréales[df_céréales['Code zone'] != 351]
df_sousalimentation = df_sousalimentation[df_sousalimentation['Code zone'] != 351 ]

In [342]:
#Ajouter une colonne 'Radical' aux deux tables df_vegetaux et df_animaux
df_vegetaux['Radical'] = 'vegetal'
df_animaux['Radical'] = 'animal'

In [343]:
#fusionner les deux tables
df_vegetaux_animaux = df_animaux.append (df_vegetaux) 


In [344]:
# Jointure avec la table population sur le Code zone 
df_vegetaux_animaux= pd.merge(df_vegetaux_animaux,df_population, on = 'Code zone' )


In [345]:
#  recréer un DataFrame avec les colonnes nécessaires
df_vegetaux_animaux= df_vegetaux_animaux[['Code Domaine_x', 'Domaine_x', 'Code zone', 'Zone_x', 'Code Élément_x',
       'Élément_x', 'Code Produit_x', 'Produit_x', 'Année_x',
       'Unité_x', 'Valeur_x','Radical', 'Valeur_y']]

In [346]:
#Changer les noms de colonnes
df_vegetaux_animaux = df_vegetaux_animaux.rename(columns= {'Code Domaine_x':'Code Domaine', 'Domaine_x':'Domaine', 'Code zone': 'Code zone', 'Zone_x': 'Zone', 'Code Élément_x': 'Code Élément',
       'Élément_x':'Élément', 'Code Produit_x':'Code Produit', 'Produit_x': 'Produit', 'Année_x': 'Année',
       'Unité_x':'Unité', 'Valeur_x':'Valeur','Radical':'Radical', 'Valeur_y':'Population'})   

In [347]:
#Population par pays
df_vegetaux_animaux['Population'] = df_vegetaux_animaux['Population'] * 1000

In [348]:
# On créé un pivot avec les éléments en colonne
df_vegetaux_animaux = df_vegetaux_animaux.pivot_table( index = [ 'Code zone', 'Zone', 'Code Produit', 'Produit', 'Année', 'Radical', 'Population'] , columns = 'Élément', values = 'Valeur', aggfunc = sum).reset_index()
df_vegetaux_animaux.head()


Élément,Code zone,Zone,Code Produit,Produit,Année,Radical,Population,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),...,Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Semences,Traitement,Variation de stock
0,1,Arménie,2511,Blé,2013,vegetal,2977000,93.0,0.0,1024.0,...,30.52,554.0,1.0,361.0,389.0,32.0,312.0,30.0,10.0,-118.0
1,1,Arménie,2513,Orge,2013,vegetal,2977000,137.0,26.0,0.0,...,0.00,198.0,0.0,9.0,0.0,15.0,189.0,14.0,7.0,0.0
2,1,Arménie,2514,Maïs,2013,vegetal,2977000,96.0,NaN,0.0,...,0.01,102.0,NaN,82.0,0.0,7.0,21.0,0.0,NaN,NaN
3,1,Arménie,2515,Seigle,2013,vegetal,2977000,1.0,NaN,1.0,...,0.02,1.0,NaN,0.0,0.0,0.0,1.0,0.0,NaN,0.0
4,1,Arménie,2516,Avoine,2013,vegetal,2977000,4.0,NaN,2.0,...,0.09,6.0,NaN,1.0,1.0,0.0,5.0,0.0,NaN,NaN


In [349]:
import sqlite3
#Faire la base de données "la faim dans le monde en 2013.db"
con = sqlite3.connect('la faim dans le monde en 2013.db')
cursor =con.cursor()


### Une table appelée population, contenant la population de chaque pays pour 2013. Elle devra contenir 4 colonnes : pays, code_pays, annee, population.


In [350]:
# Population 2013
df_population_2013 = df_population [[ 'Zone','Code zone', 'Année','Valeur']]
df_population_2013['Valeur'] = df_population_2013['Valeur'] * 1000
df_population_2013 = df_population_2013.rename (columns ={'Zone':'pays',  'Code zone':'code_pays','Année': 'annee', 'Valeur' :'population'})
df_population_2013.head()


<ipython-input-350-a610f6eee758>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_population_2013['Valeur'] = df_population_2013['Valeur'] * 1000


,pays,code_pays,annee,population
0,Afghanistan,2,2013,30552000
1,Afrique du Sud,202,2013,52776000
2,Albanie,3,2013,3173000
3,Algérie,4,2013,39208000
4,Allemagne,79,2013,82727000


In [351]:
#Pour exporter une trame de données de Python vers une base de données, nous passons par le fichier CSV. En Python, en utilisant to_csv :
df_population_2013.to_csv("exports/population.csv", index = False)



In [352]:
#Mise de population dans la base de donnée
df_population_2013.to_sql(
    name='population', # nom de la table
   con = sqlite3.connect('la faim dans le monde en 2013.db'), # connexion utilisé (ici, celle donnée par SQLAlchemy)
    index=False,  # on ne veut pas écrire les index dans la table
    if_exists='replace' # si la table existe déjà, on l'écrase
)

DatabaseError: Execution failed on sql 'DROP TABLE "population"': database is locked

### Une table appelée dispo_alim contenant pour chaque pays et pour chaque produit en 2013, les informations suivantes:\
la nature du produit (deux valeurs possibles : “animal” ou “végétal”)\
disponibilité alimentaire en tonnes\
disponibilité alimentaire en Kcal/personne/jour\
disponibilité alimentaire de protéines en g/personne/jour\
disponibilité alimentaire de matières grasses en g/personne/jour\
Elle devra contenir ces colonnes : pays, code_pays, année, produit, code_produit, origin, dispo_alim_tonnes, \dispo_alim_kcal_p_j, dispo_prot, dispo_mat_gr .


In [ ]:
df_dispo_alim = df_vegetaux_animaux[['Code zone','Zone','Année','Code Produit', 'Produit', 'Radical',  'Population' , 'Disponibilité alimentaire en quantité (kg/personne/an)',  'Disponibilité alimentaire (Kcal/personne/jour)','Disponibilité de protéines en quantité (g/personne/jour)', 'Disponibilité de matière grasse en quantité (g/personne/jour)']]

#calculer disponibilité alimentaire en tonnes
df_dispo_alim['disponibilité alimentaire en tonnes'] = df_dispo_alim['Disponibilité alimentaire en quantité (kg/personne/an)']* df_dispo_alim['Population']*365 /1000
df_dispo_alim = df_dispo_alim.rename(columns = {'Code zone': 'code_pays','Zone': 'pays','Année':  'année','Code Produit': 'code_produit', 'Produit':'produit' , 'Radical': 'origin','disponibilité alimentaire en tonnes':'dispo_alim_tonnes',  'Disponibilité alimentaire (Kcal/personne/jour)': 'dispo_alim_kcal_p_j','Disponibilité de protéines en quantité (g/personne/jour)': 'dispo_prot', 'Disponibilité de matière grasse en quantité (g/personne/jour)': 'dispo_mat_gr'  })
df_dispo_alim.drop('Population', axis='columns', inplace=True)
df_dispo_alim.drop('Disponibilité alimentaire en quantité (kg/personne/an)', axis='columns', inplace=True)

df_dispo_alim = df_dispo_alim.dropna(subset=['dispo_alim_tonnes', 'dispo_alim_kcal_p_j', 'dispo_prot', 'dispo_mat_gr'],how='all')
df_dispo_alim.head()


In [ ]:
  # Exporter le tableau sous forme de fichier CSV
df_dispo_alim.to_csv('exports/dispo_alim.csv', index=False)

In [ ]:
#Mise de population dans la base de donnée
df_dispo_alim.to_sql(
    name='dispo_alim', # nom de la table
   con = sqlite3.connect('la faim dans le monde en 2013.db'), # connexion utilisé (ici, celle donnée par SQLAlchemy)
    index=False,  # on ne veut pas écrire les index dans la table
    if_exists='replace' # si la table existe déjà, on l'écrase
)




### Une table appelée equilibre_prod contenant pour chaque pays et pour chaque produit en 2013, les quantités suivantes :\
disponibilité intérieure\
aliments pour animaux\
semences\
pertes\
transformés\
nourriture\
autres utilisations\
Elle devra contenir ces colonnes : pays, code_pays, année, produit, code_produit, dispo_int, alim_ani, semences, \pertes, transfo, nourriture, autres_utilisations.



In [ ]:
# Export du dataframe equilibre_prod
df_equilibre_prod = df_vegetaux_animaux[['Zone','Code zone', 'Année','Produit', 'Code Produit','Disponibilité intérieure', 'Aliments pour animaux', 'Semences', 'Pertes','Traitement','Nourriture','Autres utilisations (non alimentaire)']]
df_equilibre_prod = df_equilibre_prod.rename(columns =  {'Code zone':'code_pays', 'Zone':'pays', 'Code Produit' :'code_produit', 'Produit':'produit', 'Année':'année','Disponibilité intérieure': 'dispo_int', 'Aliments pour animaux':'alim_ani', 'Semences': 'semences', 'Pertes': 'pertes','Traitement': 'transfo','Nourriture':'nourriture','Autres utilisations (non alimentaire)':'autres_utilisations'})

# Conversion des unités "Millier de tonnes" en Tonnes
df_equilibre_prod.loc[:, ['dispo_int', 'alim_ani', 'semences', 'pertes', 'transfo', 'nourriture','autres_utilisations']]*=1000

df_equilibre_prod.head()


In [ ]:
#Exporter le tableau sous forme de fichier CSV.
df_equilibre_prod.to_csv('exports/equilibre_prod.csv', index= False)

In [ ]:
#Mise de population dans la base de donnée
df_equilibre_prod.to_sql(
    name='equilibre_prod', # nom de la table
   con = sqlite3.connect('la faim dans le monde en 2013.db'), # connexion utilisé (ici, celle donnée par SQLAlchemy)
    index=False,  # on ne veut pas écrire les index dans la table
    if_exists='replace' # si la table existe déjà, on l'écrase
)



### Une table appelée sous_nutrition, contenant le nombre de personnes en sous-alimentation pour chaque pays en 2013.\
Elle devra contenir 4 colonnes : pays, code_pays, année, nb_personnes.


In [ ]:
df_sousalimentation_an = df_sousalimentation.pivot_table(index = ['Code zone','Zone','Code Produit','Produit'], columns = 'Année', values = 'Valeur', aggfunc = sum).reset_index()
df_sousalimentation_an[df_sousalimentation_an['2012-2014'] == '<0.1'] = 0

# Puis on passe la colonne en type numeric
df_sousalimentation_an['2012-2014'] = pd.to_numeric(df_sousalimentation_an['2012-2014'] , errors='coerce')



In [ ]:
#Une table appelée sous_nutrition, contenant le nombre de personnes en sous-alimentation pour chaque pays en 2013. Elle devra contenir 4 colonnes : pays, code_pays, année, nb_personnes.
df_sous_nutrition = df_sousalimentation_an[['Zone','Code zone','2012-2014']]
df_sous_nutrition ['annee'] = 2013
df_sous_nutrition['2012-2014'] = df_sous_nutrition['2012-2014'] * 1000000

df_sous_nutrition= df_sous_nutrition.rename(columns={'Code zone':' code_pays', 'Zone': 'pays','2012-2014':'nb_personnes'})


In [ ]:
df_sousalimentation_an[df_sousalimentation_an['2012-2014'] == '<0.1'] = 0

# Puis on passe la colonne en type numeric
df_sousalimentation_an['2012-2014'] = df_sousalimentation_an['2012-2014'].apply(pd.to_numeric)



In [ ]:
df_sous_nutrition

In [ ]:
#Exporter le tableau sous forme de fichier CSV.
df_sous_nutrition.to_csv('exports/sous_nutrition.csv' , index = False)

In [ ]:
#Mise de population dans la base de donnée
df_sous_nutrition.to_sql(
    name='sous_nutrition', # nom de la table
   con = sqlite3.connect('la faim dans le monde en 2013.db'), # connexion utilisé (ici, celle donnée par SQLAlchemy)
    index=False,  # on ne veut pas écrire les index dans la table
    if_exists='replace' # si la table existe déjà, on l'écrase
)
